In [1]:
import os
import zipfile
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import img_to_array, load_img

In [2]:
# Get the Horse or Human training dataset
!wget -q -P /content/ https://storage.googleapis.com/tensorflow-1-public/course2/week3/horse-or-human.zip

# Get the Horse or Human validation dataset
!wget -q -P /content/ https://storage.googleapis.com/tensorflow-1-public/course2/week3/validation-horse-or-human.zip

In [3]:
#unzip training set
train_local_zip = './horse-or-human.zip'
zip_ref = zipfile.ZipFile(train_local_zip, 'r')
zip_ref.extractall('./horse-or-human')

#unzip validation set
val_local_zip = './validation-horse-or-human.zip'
zip_ref = zipfile.ZipFile(val_local_zip, 'r')
zip_ref.extractall('./validation-horse-or-human')

In [4]:
import os

#training
train_horse_dir = os.path.join('./horse-or-human/horses')
train_human_dir = os.path.join('./horse-or-human/humans')

#validation
validation_horse_dir = os.path.join('./validation-horse-or-human/horses')
validation_human_dir = os.path.join('./validation-horse-or-human/humans')

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255)
val_datagen = ImageDataGenerator(rescale=1/255)

train_gen = train_datagen.flow_from_directory(
    './horse-or-human/',
    target_size=(300,300),
    batch_size=128,
    class_mode='binary'
)
val_gen = val_datagen.flow_from_directory(
    './validation-horse-or-human',
    target_size=(300,300),
    batch_size=32,
    class_mode = 'binary'
)

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


Here are the additional changes: We add the transfer model.

In [6]:
#download the pretrained model
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

--2023-08-31 05:11:13--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.128.128, 74.125.143.128, 173.194.69.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.128.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M  26.3MB/s    in 3.4s    

2023-08-31 05:11:17 (24.8 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



In [7]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

# Create an instance of the inception model from the local pre-trained weights
local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [8]:
pre_trained_model = InceptionV3(
    input_shape=(150,150,3),
    include_top=False,
    weights=None
)
#we use our saved weights
pre_trained_model.load_weights(local_weights_file)

#make all layers in pre-trained model non-trainable
for layer in pre_trained_model.layers:
  layer.trainable=False

Now we 'glue' it to our own model. We get the last output of the pre-trained model, which will be the input for our own model.

In [9]:
last_desired_layer = pre_trained_model.get_layer('mixed7')
last_output=last_desired_layer.output

In [10]:
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop

def create_final_model(pre_trained_model, last_output):

  x = layers.Flatten()(last_output)
  x = layers.Dense(1024, activation = 'relu')(x)
  x = layers.Dense(1, activation = 'sigmoid')(x)

  model = Model(pre_trained_model.input, x)

  model.compile(
      loss = 'binary_crossentropy',
      optimizer = RMSprop(learning_rate=0.001),
      metrics=['accuracy']
  )
  return model

In [11]:
model = create_final_model(pre_trained_model, last_output)

In [ ]:
model.fit(
    train_gen,
    steps_per_epoch=8,
    epochs=15,
    verbose=1,
    validation_data = val_gen,
    validation_steps=8)